Rename the files in a Rearm visit following the naming convention. 

The goal is to make the correction of "typical" human errors in the naming of the files... 

The naming convention recalled below is extensively described in [notebooks/checkFilesInVisit.ipynb](checkFilesInVisit.ipynb)
```
<project>_<participant>_<date>_<visit>_<record>_<specific>.<extension>
```

The corrections to be made are:
- Double extension to a file
- token `<task>` placed  at the end of the file name (it should be after `<visit>`)
- token {`_mau_np`, ..., `_sau_p`} in file name (it should be `_np_mau`, ..., `_p_sau`)
- combination of all the above errors
- token `<task>` named as `_Accelerometry` (it should be `_a`)

The function `rename_files` is provided to rename the files in a Rearm visit following the naming convention: 

- It outputs a log file in the visit directory. You have to remove this file before running the function again.


In [ ]:
# this should be the set to false for production use
# and set to true for testing (default) but can be already set to false from outside (e.g., by the test script)

if "doRunTests" not in globals():
    doRunTests = True
    pathToData = "dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1"

In [ ]:
# rename files

import os
import glob


def rename_file(full_fname, full_new_fname):
    """rename the file fname to new_fname"""
    # comment out the following line to avoid renaming the files (for testing)
    os.rename(full_fname, full_new_fname)


def change_substring_in_filename(fullFileName, substring, new_substring):
    """change a substring in the file name"""
    fname = os.path.splitext(os.path.basename(fullFileName))[0]
    fext = os.path.splitext(os.path.basename(fullFileName))[1]
    fpath = os.path.dirname(fullFileName)
    new_fname = fname.replace(substring, new_substring)

    # if fname and new_fname are different, rename the file
    full_fname = os.path.join(fpath, fname + fext)
    full_new_fname = os.path.join(fpath, new_fname + fext)
    if full_fname != full_new_fname:
        rename_file(full_fname, full_new_fname)

    return full_fname, full_new_fname


def fname_without_double_extension(fname, fext):
    """return the file name without double extension"""
    if fname.endswith(fext):
        return fname.replace(fext, "")
    return fname


def remove_double_extension(fullFileName):
    """remove a possible double extension in fileName"""
    fname = os.path.splitext(os.path.basename(fullFileName))[0]
    fext = os.path.splitext(os.path.basename(fullFileName))[1]
    fpath = os.path.dirname(fullFileName)
    new_fname = fname_without_double_extension(fname, fext)

    full_fname = os.path.join(fpath, fname + fext)
    full_new_fname = os.path.join(fpath, new_fname + fext)
    if full_fname != full_new_fname:
        rename_file(full_fname, full_new_fname)

    return full_fname, full_new_fname


def reorder_tokens_in_file_name(fullFileName, task_token):
    """reorder the tokens in the file name"""
    fname = os.path.splitext(os.path.basename(fullFileName))[0]
    fext = os.path.splitext(os.path.basename(fullFileName))[1]
    fpath = os.path.dirname(fullFileName)
    tokens = fname.split("_")
    new_fname = fname
    # change the token order to Rearm_Patient_Date_Visit_Task_...
    # if the last token is the task token
    if tokens[-1] == task_token and tokens[4] != task_token:
        # insert the last token after the visit token (4th token)
        new_fname = (
            tokens[0]
            + "_"
            + tokens[1]
            + "_"
            + tokens[2]
            + "_"
            + tokens[3]
            + "_"
            + tokens[-1]
        )
        # if there are more than 5 tokens,
        # add the extra tokens to the new file name (omit the last token)
        if len(tokens) > 5:
            for i in range(4, len(tokens) - 1):
                new_fname = new_fname + "_" + tokens[i]

    # if fname and new_fname are different, rename the file
    full_fname = os.path.join(fpath, fname + fext)
    full_new_fname = os.path.join(fpath, new_fname + fext)
    if full_fname != full_new_fname:
        rename_file(full_fname, full_new_fname)

    return full_fname, full_new_fname


def print_output(fname1, fname2, pathToVisit):
    """print the original and new file names"""
    relativePath1 = os.path.relpath(fname1, pathToVisit)
    relativePath2 = os.path.relpath(fname2, pathToVisit)
    relativePath2 = os.path.normpath(relativePath2)
    if fname1 != fname2:
        print("    " + relativePath1)
        print(" -> " + relativePath2)
    # else:
    #     print("    " + fname1 + " -> unchanged")
    # print(" ")


if doRunTests:
    relativePath = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1/ReArm_C1P02_20210306_V1_Circle"
    task_token = "c"

    # relativePath = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1/ReArm_C1P02_20210306_V1_Reaching"
    # task_token = "r"

    absolutePath = os.path.abspath(relativePath)

    print("Reorder tokens in file name")
    absFilePathList = glob.glob(absolutePath + "/*")
    for absFileName in absFilePathList:
        fullFile = os.path.join(absolutePath, os.path.basename(absFileName))
        f_in_r, f_out_r = reorder_tokens_in_file_name(fullFile, task_token)
        print_output(f_in_r, f_out_r, absolutePath)

    print("Remove double extension in file names")
    absFilePathList = glob.glob(absolutePath + "/*")
    for absFileName in absFilePathList:
        fullFile = os.path.join(absolutePath, os.path.basename(absFileName))
        f_in, f_new = remove_double_extension(fullFile)
        print_output(f_in, f_new, absolutePath)

In [ ]:
original_str = "Hello, world!"
substring1 = "Corld"
substring2 = "Python"

new_str = original_str.replace(substring1, substring2)
print(new_str)  # Output: Hello, Python!

In [ ]:
import logging


def list_all_absFilePath(absVisitPath):
    """
    Sets the list of absolute path to all files in the visit directory.
    """
    allFiles = glob.glob(os.path.join(absVisitPath, "**/*"), recursive=True)
    allFiles = [f for f in allFiles if not os.path.isdir(f)]
    allFiles = [os.path.abspath(f) for f in allFiles]
    return allFiles


def remove_double_extension_in_all_files(absVisitPath):
    """
    Remove double extension in all file names in the visit directory.
    """
    absFilePathList = list_all_absFilePath(absVisitPath)
    for absFileName in absFilePathList:
        f_in, f_new = remove_double_extension(absFileName)
        log_output(f_in, f_new, absVisitPath)


def rename_task_token_in_all_files(absVisitPath):
    """
    Rename task token in all file names in the visit directory.
    """
    absFilePathList = list_all_absFilePath(absVisitPath)
    for absFileName in absFilePathList:
        if "_Accelerometry" in os.path.dirname(absFileName):
            f_in_a, f_out_a = change_substring_in_filename(
                absFileName, "Accelerometry", "ac"
            )
            log_output(f_in_a, f_out_a, absVisitPath)


def reorder_tokens_sau_mau_before_p_np(absFileName):
    full_fpath = os.path.dirname(absFileName)
    file_name = os.path.basename(absFileName)

    # Reorder tokens in the file name
    new_file_name = file_name
    if "_np_mau" in new_file_name:
        new_file_name = new_file_name.replace("_np_mau", "_mau_np")
    if "_p_mau" in new_file_name:
        new_file_name = new_file_name.replace("_p_mau", "_mau_p")
    if "_np_sau" in new_file_name:
        new_file_name = new_file_name.replace("_np_sau", "_sau_np")
    if "_p_sau" in new_file_name:
        new_file_name = new_file_name.replace("_p_sau", "_sau_p")

    full_new_fname = os.path.join(full_fpath, new_file_name)
    rename_file(absFileName, full_new_fname)

    return absFileName, full_new_fname


def reorder_tokens_in_all_files(absVisitPath):
    """
    Reorder tokens in all file names in the visit directory.
    """
    absFilePathList = list_all_absFilePath(absVisitPath)
    for absFileName in absFilePathList:

        # Reorder the task token in the file name
        full_fpath = os.path.dirname(absFileName)
        f_out_r = absFileName
        if "_Reaching" in full_fpath:
            f_in_r, f_out_r = reorder_tokens_in_file_name(absFileName, "r")
        if "_Circle" in full_fpath:
            f_in_r, f_out_r = reorder_tokens_in_file_name(absFileName, "c")
        if "_Accelerometry" in full_fpath:
            f_in_r, f_out_r = reorder_tokens_in_file_name(absFileName, "ac")
        if "_Armeo" in full_fpath:
            f_in_r, f_out_r = reorder_tokens_in_file_name(absFileName, "a")

        # Reorder the sau-mau, p-np tokens in the file name
        f_in_sau, f_out_sau = reorder_tokens_sau_mau_before_p_np(f_out_r)
        if absFileName != f_out_sau:
            log_output(absFileName, f_out_sau, absVisitPath)


def log_output(f_in, f_out, pathToVisit):
    """
    Log the output of file renaming.
    """
    relativePath1 = os.path.relpath(f_in, pathToVisit)
    relativePath2 = os.path.relpath(f_out, pathToVisit)
    relativePath2 = os.path.normpath(relativePath2)

    if f_in != f_out:
        # shift_spaces is for nice output formatting in the log file
        shift_spaces = "                                   "
        message = f"  {relativePath1} \n{shift_spaces}{relativePath2}"
        logging.info(message)


def renameFilesInVisit(visitPath):
    """
    Rename all files in the visit directory.

    Input arguments:
    - visitPath: the path to the visit directory

    Output:
    - a log file (in the visit directory) with the details of the renaming process
    """

    absVisitPath = os.path.abspath(visitPath)
    checkLog_fname = "file_renaming.log"
    full_checkLog_fname = os.path.join(absVisitPath, checkLog_fname)

    # Check if the visit was already renamed (log file exists)
    if os.path.isfile(full_checkLog_fname):
        print("    Renaming process already done: '%s' found" % checkLog_fname)
    else:
        # Create the log file
        logging.basicConfig(
            filename=full_checkLog_fname,
            level=logging.INFO,
            format="%(asctime)s - %(levelname)s - %(message)s",
            force=True,  # remove previous handlers and set the new one
        )

        logging.info("Starting file renaming process in directory")

        logging.info("Remove double extension in all file names in the visit directory")
        remove_double_extension_in_all_files(absVisitPath)

        logging.info("Rename task token in all file names in the visit directory")
        rename_task_token_in_all_files(absVisitPath)

        logging.info("Reorder tokens in all file names in the visit directory")
        reorder_tokens_in_all_files(absVisitPath)

        logging.info("File renaming process completed")
        print(
            "    File renaming process completed: see '%s' for details" % checkLog_fname
        )


if doRunTests:
    visitPath = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1/"
    visitPath = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210419_V2/"
    # visitPath = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210715_V3/"

    renameFilesInVisit(visitPath)

# Testing the function `renameFilesInVisit`

To test the function `renameFilesInVisit`, I did some manual tests, all of which passed: 

- Double extension to a file
- token `<task>` placed  at the end of the file name (it should be after `<visit>`)
- token {`_mau_np`, ..., `_sau_p`} in file name (it should be `_np_mau`, ..., `_p_sau`)
- combination of all the above errors
- token `<task>` named as `_Accelerometry` (it should be `_a`)

